### Load environment variables

In [24]:
from dotenv import load_dotenv
import os

# 加载 .env 文件
load_dotenv(override=True)

# 访问环境变量
YOUTUBE_API_KEY = os.getenv('YOUTUBE_API_KEY')
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
S3_BUCKET_NAME = os.getenv('S3_BUCKET_NAME')

### Youtube API

In [44]:
from googleapiclient.discovery import build

def search_youtube_videos(query, max_results=50):
    """
    使用 YouTube Data API 搜索视频，支持分页
    :param query: 搜索关键字
    :param max_results: 需要返回的总视频数量
    :return: 视频的标题和完整链接
    """
    youtube = build("youtube", "v3", developerKey=YOUTUBE_API_KEY)
    video_results = []
    next_page_token = None
    
    while len(video_results) < max_results:
        # 计算本次请求需要的结果数
        remaining = min(50, max_results - len(video_results))
        
        # 执行搜索请求
        request = youtube.search().list(
            part="snippet",
            q=query,
            maxResults=remaining,
            type="video",
            pageToken=next_page_token
        )
        response = request.execute()
        
        # 提取视频信息
        for item in response.get('items', []):
            video_id = item['id']['videoId']
            title = item['snippet']['title']
            url = f"https://www.youtube.com/watch?v={video_id}"
            video_results.append({'title': title, 'video_id': video_id, 'url': url})
        
        # 获取下一页的 token
        next_page_token = response.get('nextPageToken')
        
        # 如果没有下一页，就退出循环
        if not next_page_token:
            break
        
        print(f"已获取 {len(video_results)} 个视频...")
    
    print(f"总共获取 {len(video_results)} 个视频")
    return video_results

In [4]:
query = "No Limit Texas Hold'em poker strategy"

In [13]:
results = search_youtube_videos(query,10)
results

[{'title': 'This Easy No Limit Hold&#39;em Strategy SKYROCKETED My Winnings',
  'video_id': 'eAJG5q-_OoU',
  'url': 'https://www.youtube.com/watch?v=eAJG5q-_OoU'},
 {'title': 'Win More at Poker - Easy Strategy for Hold&#39;em Starting Hands',
  'video_id': 'aJ3rfNH-yGY',
  'url': 'https://www.youtube.com/watch?v=aJ3rfNH-yGY'},
 {'title': 'How to Play Poker for Beginners | PokerStars Learn',
  'video_id': 'pSRGErzzIo4',
  'url': 'https://www.youtube.com/watch?v=pSRGErzzIo4'},
 {'title': 'How to Play Limit Hold&#39;em',
  'video_id': '-cu4kQhARJI',
  'url': 'https://www.youtube.com/watch?v=-cu4kQhARJI'},
 {'title': 'How to Play No Limit Texas Hold’em (5 Beginner Poker Strategies)',
  'video_id': 'bcKjZq43NjE',
  'url': 'https://www.youtube.com/watch?v=bcKjZq43NjE'},
 {'title': 'TOP 10 Cash Game TIPS! [Master The Fundamentals]',
  'video_id': 'f-MnvLLYqPE',
  'url': 'https://www.youtube.com/watch?v=f-MnvLLYqPE'},
 {'title': '9 TEXAS HOLD&#39;EM Poker Tips For Beginners (Just Do This!)',
 

In [37]:
from youtube_transcript_api import YouTubeTranscriptApi
import json

def get_transcript(video_id):
    """
    使用 youtube-transcript-api 获取视频的转录文本
    :param video_id: YouTube 视频 ID
    :return: 转录文本的 JSON 字符串
    """
    try:
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
        # 直接返回 JSON 字符串
        return json.dumps(transcript_list)
    except Exception as e:
        print(f"获取字幕失败: {str(e)}")
        return "[]"  # 如果获取失败，返回空数组的字符串表示

In [15]:
for video in results:
    transcript = get_transcript(video['video_id'])
    print(transcript)


[{'text': "what's going on guys today I'm going to", 'start': 0.0, 'duration': 4.02}, {'text': 'walk you step by step through the simple', 'start': 1.8, 'duration': 4.62}, {'text': 'poker strategy that will quickly boost', 'start': 4.02, 'duration': 4.26}, {'text': "your winnings let's jump into it all", 'start': 6.42, 'duration': 3.599}, {'text': 'right guys do you play small stakes', 'start': 8.28, 'duration': 5.1}, {'text': "poker if you're like 95 of people you", 'start': 10.019, 'duration': 5.881}, {'text': 'probably do this would be Stakes like a', 'start': 13.38, 'duration': 4.319}, {'text': 'one dollar two dollar cash game at a', 'start': 15.9, 'duration': 4.68}, {'text': 'casino a 20 tournament 50 tournament', 'start': 17.699, 'duration': 5.281}, {'text': 'maybe a 50 buy-in cash game on the', 'start': 20.58, 'duration': 4.5}, {'text': 'internet you guys get the idea these are', 'start': 22.98, 'duration': 3.84}, {'text': 'the stakes that the vast majority of', 'start': 25.08, 

### Store information to DynamoDB

In [6]:
AWS_ACCESS_KEY_ID

'poker_db'

In [25]:
import boto3
from botocore.exceptions import ClientError


# 初始化 DynamoDB 客户端
dynamodb = boto3.resource(
    'dynamodb',
    aws_access_key_id=AWS_ACCESS_KEY_ID,  # 替换为你的 Access Key
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,  # 替换为你的 Secret Key
    region_name='us-east-1'  # 替换为你的区域
)

# 表名
table_name = "youtube_poker_db"

In [27]:
table = dynamodb.Table(table_name)
table.table_status

'ACTIVE'

In [28]:
def insert_video_metadata(table_name,video_id, title, url, transcript):
    """
    将视频元数据插入到 DynamoDB 表中
    :param video_id: 视频的唯一 ID
    :param title: 视频标题
    :param url: 视频 URL
    :param transcript: 视频转录文本
    """
    table = dynamodb.Table(table_name)
    try:
        table.put_item(
            Item={
                "VideoID": video_id,  # Partition Key
                "Title": title,
                "URL": url,
                "Transcript": transcript
            }
        )
        print(f"Video {video_id} metadata inserted successfully.")
    except ClientError as e:
        print(f"Error inserting metadata: {e.response['Error']['Message']}")

In [40]:
def insert_video_metadata_batch(query, table_name, max_results=10):
    """
    批量插入视频元数据到 DynamoDB
    """
    success_count = 0
    error_count = 0
    
    try:
        # 首先验证 AWS 认证
        table = dynamodb.Table(table_name)
        table.table_status  # 测试连接
        
        # 获取视频列表
        videos = search_youtube_videos(query, max_results)
        print(f"找到 {len(videos)} 个视频")
        
        for video in videos:
            try:
                # 获取转录文本
                transcript_text = get_transcript(video['video_id'])
                
                # 插入数据
                insert_video_metadata(
                    table_name,
                    video['video_id'],
                    video['title'],
                    video['url'],
                    transcript_text  # 现在传入的是字符串
                )
                success_count += 1
                print(f"成功插入视频: {video['title']}")
                
            except Exception as e:
                error_count += 1
                print(f"处理视频时出错: {str(e)}")
                
        print(f"\n插入总结:")
        print(f"成功: {success_count}")
        print(f"失败: {error_count}")
        
    except Exception as e:
        print(f"严重错误: {str(e)}")
        print("请检查 AWS 认证设置")
        
    return success_count, error_count

In [45]:
insert_video_metadata_batch(query,table_name,200)

已获取 50 个视频...
已获取 100 个视频...
已获取 150 个视频...
已获取 200 个视频...
总共获取 200 个视频
找到 200 个视频
Video eAJG5q-_OoU metadata inserted successfully.
成功插入视频: This Easy No Limit Hold&#39;em Strategy SKYROCKETED My Winnings
Video aJ3rfNH-yGY metadata inserted successfully.
成功插入视频: Win More at Poker - Easy Strategy for Hold&#39;em Starting Hands
Video pSRGErzzIo4 metadata inserted successfully.
成功插入视频: How to Play Poker for Beginners | PokerStars Learn
Video -cu4kQhARJI metadata inserted successfully.
成功插入视频: How to Play Limit Hold&#39;em
Video bcKjZq43NjE metadata inserted successfully.
成功插入视频: How to Play No Limit Texas Hold’em (5 Beginner Poker Strategies)
Video noKz7aCfwNM metadata inserted successfully.
成功插入视频: 9 TEXAS HOLD&#39;EM Poker Tips For Beginners (Just Do This!)
Video GAoR9ji8D6A metadata inserted successfully.
成功插入视频: 🎒 📈 How to Play Poker - Texas Hold&#39;em Rules Made Easy
Video f-MnvLLYqPE metadata inserted successfully.
成功插入视频: TOP 10 Cash Game TIPS! [Master The Fundamentals]
Video EKB-

(200, 0)

In [42]:
# 验证数据是否成功插入到 DynamoDB
def verify_dynamodb_insertion(table_name):
    """
    验证数据是否成功插入到 DynamoDB 表中
    :param table_name: DynamoDB 表名
    """
    table = dynamodb.Table(table_name)
    try:
        # 获取表中的所有项目
        response = table.scan()
        items = response['Items']
        
        print(f"\n=== DynamoDB 表 {table_name} 验证结果 ===")
        print(f"总记录数: {len(items)}")
        
        if items:
            print("\n示例记录:")
            print(json.dumps(items[0], indent=2, ensure_ascii=False))
            
            # 检查字段完整性
            fields = ['VideoID', 'Title', 'URL', 'Transcript']
            for field in fields:
                coverage = sum(1 for item in items if field in item)
                print(f"\n{field} 字段覆盖率: {coverage}/{len(items)}")
                
        return len(items)
    
    except ClientError as e:
        print(f"错误: {e.response['Error']['Message']}")
        return 0

# 执行验证
record_count = verify_dynamodb_insertion(table_name)


=== DynamoDB 表 youtube_poker_db 验证结果 ===
总记录数: 10

示例记录:
{
  "Title": "9 TEXAS HOLD&#39;EM Poker Tips For Beginners (Just Do This!)",
  "Transcript": "[{\"text\": \"what's going on guys Nathan here and\", \"start\": 0.0, \"duration\": 3.3}, {\"text\": \"today I'm going to give you the top nine\", \"start\": 1.8, \"duration\": 4.14}, {\"text\": \"things that you will never see a good\", \"start\": 3.3, \"duration\": 5.28}, {\"text\": \"Texas Hold'em poker player doing let's\", \"start\": 5.94, \"duration\": 4.44}, {\"text\": \"jump right into it all right guys so in\", \"start\": 8.58, \"duration\": 4.08}, {\"text\": \"my 10 plus years as a professional in\", \"start\": 10.38, \"duration\": 4.139}, {\"text\": \"this game I can tell you that I have\", \"start\": 12.66, \"duration\": 4.619}, {\"text\": \"never seen a good Texas poker player\", \"start\": 14.519, \"duration\": 5.281}, {\"text\": \"doing any of the nine things on this\", \"start\": 17.279, \"duration\": 4.561}, {\"text\": 